In [ ]:
# 1. Install Ollama
!curl -fsSL https://ollama.com/install.sh | sh

# 2. Start Ollama server in the background
import subprocess
import time
import os

os.environ['PATH'] += ':/usr/local/bin'

with open("ollama_server.log", "w") as log_file:
    process = subprocess.Popen(["ollama", "serve"], stdout=log_file, stderr=log_file)

time.sleep(10) # Wait for server
!ollama pull mistral:7b-instruct-v0.2-q4_0
!pip install ollama

>>> Installing ollama to /usr/local
>>> Downloading Linux amd64 bundle
######################################################################## 100.0%
>>> Creating ollama user...
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.



In [ ]:
from google.colab import drive
import os

drive.mount('/content/drive')

# Change these to your actual dissertation folder paths
BASE_DIR = "/content/drive/MyDrive/Dissertation"
TRANSCRIPT_DIR = os.path.join(BASE_DIR, "transcripts_txt")
SUMMARY_DIR = os.path.join(BASE_DIR, "summaries_json")

os.makedirs(TRANSCRIPT_DIR, exist_ok=True)
os.makedirs(SUMMARY_DIR, exist_ok=True)



Mounted at /content/drive


In [ ]:
from google.colab import drive
import os

drive.mount('/content/drive')

# Change these to your actual dissertation folder paths
BASE_DIR = "/content/drive/MyDrive/Dissertation"
TRANSCRIPT_DIR = os.path.join(BASE_DIR, "transcripts_txt")
SUMMARY_DIR = os.path.join(BASE_DIR, "summaries_json")

os.makedirs(TRANSCRIPT_DIR, exist_ok=True)
os.makedirs(SUMMARY_DIR, exist_ok=True)



In [13]:
import ollama
import re
import json
import time

# Full 60 Topics (Make sure all 60 are here)
topics = [
   "Meeting 01: Project Kick-off: Scope definition for Route Creation and UI requirements.",
   "Meeting 02: Business Requirements: Defining 'Route Base Purpose' and mandatory attributes."
  ]

with open(os.path.join(BASE_DIR, "promotMeeting.txt"), 'r') as f:
    base_context = f.read()


In [17]:
def generate_meeting_files(i, topic):
    meeting_id = f"Meeting_{i+1:02d}"
    print(f"--- Processing {meeting_id} ---")

    # STEP 1: GENERATE TRANSCRIPT (Conversational Tone)
    # We remove the technical metadata from this prompt to make the speech natural
    t_prompt = f"""{base_context}
    TASK: Write a 1500-word verbatim transcript for {topic}.

    CRITICAL INSTRUCTION FOR CLOSING:
    At the end of the meeting, Sarah (PO) should wrap up the session naturally.
    She should assign at least 3 specific technical tasks to Vikram, Anjali, and Priya
    as part of her closing remarks, speaking in full, conversational sentences.

    DO NOT use bullet points, numbered lists, or labels like "Issue Type" or "Due Date"
    in the dialogue. Have her speak like a real project manager.
    DO NOT generate JSON yet."""

    t_res = ollama.generate(model='mistral:7b-instruct-v0.2-q4_0', prompt=t_prompt)
    transcript = t_res['response'].strip()

    # Save Clean Transcript
    with open(f"{TRANSCRIPT_DIR}/{meeting_id}_transcript.txt", "w", encoding="utf-8") as f:
        f.write(transcript)

    # STEP 2: STRICT DATA EXTRACTION (Detailed Summary & All Keys)
    # This step handles the technical formatting that we kept out of the transcript
    s_prompt = f"""
    Read the transcript below and create a highly detailed JSON summary.
    YOU MUST FILL EVERY KEY. Provide comprehensive sentences for the summary.

    TRANSCRIPT:
    {transcript}

    JSON FORMAT TO FOLLOW (FILL ALL FIELDS):
    {{
      "meeting_id": "{meeting_id}",
      "project": "Enovia Route Automation",
      "participants": [
          {{"name": "Sarah", "role": "Product Owner"}},
          {{"name": "Vikram", "role": "Senior Enovia Engineer"}},
          {{"name": "Anjali", "role": "Assistant Software Engineer"}},
          {{"name": "Priya", "role": "Quality Engineer"}},
          {{"name": "Mark", "role": "Senior Quality Engineer"}},
          {{"name": "David", "role": "Business Analyst"}}
      ],
      "agenda": ["List at least 3 specific technical topics discussed in the dialogue"],
      "risk_factors": ["List at least 2 technical risks mentioned by the team"],
      "summary": [
          "Provide 5-7 detailed sentences explaining the specific technical decisions made."
      ],
      "action_items": [
          {{
            "summary": "Short task title",
            "description": "Detailed technical instruction based on what Sarah said",
            "assignee": "Name of the person assigned",
            "issue_type": "Story/Task/Bug",
            "story_points": 3,
            "due_date": "2024-12-25"
          }}
      ]
    }}
    RETURN ONLY THE JSON BLOCK."""

    s_res = ollama.generate(model='mistral:7b-instruct-v0.2-q4_0', prompt=s_prompt, options={"temperature": 0.1})

    # Extraction and Saving
    json_match = re.search(r'\{.*\}', s_res['response'], re.DOTALL)
    if json_match:
        try:
            valid_json = json.loads(json_match.group())
            with open(f"{SUMMARY_DIR}/{meeting_id}_summary.json", "w", encoding="utf-8") as f:
                json.dump(valid_json, f, indent=4)
            print(f"Success: Full JSON created with all keys.")
        except Exception as e:
            print(f"Error: Invalid JSON structure. {e}")

In [18]:

# Run the loop
for i, topic in enumerate(topics):
    generate_meeting_files(i, topic)

--- Processing Meeting_01 ---
Success: Full JSON created with all keys.
--- Processing Meeting_02 ---
Success: Full JSON created with all keys.
